In [2]:
import csv
import pandas as pd

# FROM CARD OPEN TO LAST MONTH OF TXN CUSTOMER SURVIVAL
(before that used sas to find debit cards)

In [40]:
file = open('all_debit_cards_base_30.12.21_270_days_gone.csv')
csv_data = csv.reader(file,delimiter = ';')

In [41]:
lines = list(csv_data)
lines[0]

['CUSTOMER_RK',
 'AGREEMENT_RK',
 'CONTRACT_ID',
 'CARD_ACCOUNT_NUM',
 'CARD_NUM',
 'CARD_TYPE_CD',
 'CARD_TYPE_NM',
 'OPEN_DT',
 'OPEN_MDY',
 'CLOSE_DT',
 'CLOSE_MDY',
 'STATUS_CD',
 'PRODUCT_NAME',
 'CLOSED_ON_NTH_MONTH',
 'LIVING_THRU_CENS',
 'MIN_of_TRANSACTION_DT',
 'MAX_of_TRANSACTION_DT',
 'MAX_TXN_DT',
 'CARD_FACT_USAGE_M',
 'STARTED_USING',
 'SURVIVED_FLG']

In [42]:
min_open = 0
max_txn = 0
break_count = 0
break_months = []
alive_flg = 0

lines[0].extend(['BREAK_MONTHS', 'BREAK_COUNT', 'MIN_OPEN', 'MAX_TXN', 'ALIVE_FLG'])

for i in range(1, len(lines)):
           
    if lines[i][0] != lines[i-1][0]:
        
        break_count = 0
        break_months = []
        alive_flg = 0
        
        min_open = int(lines[i][8])
        
        if lines[i][17] != '':
            max_txn = int(lines[i][17])
        else:
            max_txn = int(lines[i][8])
    
    else:

        if int(lines[i][8]) > max_txn:
            break_months.append(int(round((int(lines[i][8]) - max_txn) / 30, 0)))
            break_count += 1
        
            
        if lines[i][17] != '' and int(lines[i][17]) > max_txn:
            max_txn = int(lines[i][17])
        
    
    if lines[i][-1] == '1':
        alive_flg = 1
        
    lines[i].append(break_months)
    lines[i].append(break_count)
    lines[i].append(min_open)
    lines[i].append(max_txn)
    lines[i].append(alive_flg)

In [27]:
# закрывает ли карту до перерыва

min_open = 0
max_txn = 0
max_close = 0


#yes - zakryl, no - ne zakryl kartu
count_yes = []
count_no = []
count_same_m = []

for i in range(1, len(lines)):
           
    if lines[i][0] != lines[i-1][0]:
        
        
        min_open = int(lines[i][8])
                
        try:
            max_close = int(lines[i][10])
        except:
            max_close = 'Active'
        
        if lines[i][17] != '':
            max_txn = int(lines[i][17])        
    
    else:
                   
        if max_close != 'Active':
            
            if int(lines[i][8]) > max_close and int(lines[i][8]) > max_txn:
                count_yes.append(i)
            elif int(lines[i][8]) < max_close and int(lines[i][8]) > max_txn:
                count_no.append(i)
            elif int(lines[i][8]) == max_close and int(lines[i][8]) > max_txn:
                count_same_m.append(i)
                       
            if lines[i][10] == '':
                max_close = 'Active'
            elif int(lines[i][10]) > max_close:
                max_close = int(lines[i][10])
                
        if lines[i][17] != '' and int(lines[i][17]) > max_txn:
                    max_txn = int(lines[i][17])

In [28]:
count_yes[:10]

[19, 37, 45, 51, 64, 74, 82, 94, 102, 191]

In [29]:
count_no[:10]

[3, 21, 25, 69, 73, 86, 87, 88, 171, 210]

In [43]:
df = pd.DataFrame(lines)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.to_csv('lines_all_client_cards_30.12.21_270_days_gone.csv', index = False)

## after that used SAS to keep only needed information

# SURVIVAL ANALYSIS FOR DEBIT CARD CUSTOMERS

In [59]:
file = open('to_compute_hazards_2018.csv')
csv_data = csv.reader(file,delimiter = ';')
lines = list(csv_data)

In [60]:
survived_to = []
for i in range(1,len(lines)):

    try:
        survived_to.append(int(lines[i][-2]))
    except:
        pass
    
gone_on = []
for i in range(1, len(lines)):
    try:
        gone_on.append(int(lines[i][-3]))
    except:
        pass

In [61]:
survived_to_dict = {}
for x in range(200):
    count_x = 0
    for item in survived_to:
        if item>=x:
            count_x += 1
        else:
            pass
    survived_to_dict[int('{}'.format(x))] = count_x

In [62]:
gone_on_dict = {}
for i in range(1, 200):
    gone_on_dict[int('{}'.format(i))] = gone_on.count(i)

In [63]:
df = pd.DataFrame.from_dict(survived_to_dict, orient='index')
df.to_excel('survive_to.xlsx', sheet_name='survive_to')

df2 = pd.DataFrame.from_dict(gone_on_dict, orient='index')
df2.to_excel('gone_on.xlsx', sheet_name='gone_on')

# RETURN RATE

In [147]:
# 22615 01.12.21

In [12]:
file = open('to_compute_return_rate_all_client_cards_21.12.21_180_days_gone.csv')
csv_data = csv.reader(file,delimiter = ';')

In [13]:
lines = list(csv_data)
lines[1]

['9529264', '[1]', '1', '20120', '22585', '82', '81', '0']

In [14]:
lines[0]

['CUSTOMER_RK',
 'BREAK_MONTHS',
 'BREAK_COUNT',
 'MIN_OPEN',
 'MAX_TXN',
 'CLOSED_ON_NTH_MONTH',
 'LIVING_THRU_CENS',
 'ALIVE_FLG']

In [15]:
returned_on = []
for i in range(1,len(lines)):
    if lines[i][1] != '[]':
        returned_on.extend(lines[i][1].replace('[','').replace(']','').split(', '))


In [16]:
return_on_2 = list(map(lambda x: int(x), returned_on))

In [17]:
return_set = set()
for item in return_on_2:
    return_set.add(item)
return_set = list(return_set)
return_set.sort()
    
return_dict = {}
for item in return_set:
    return_dict[item] = return_on_2.count(item)

In [18]:
idet_since_gone = []

for i in range(1, len(lines)):
    if lines[i][1] == '[]' and lines[i][-1] == '0':
        idet_since_gone.append(int(round((22615 - int(lines[i][4])) / 30)))

In [19]:
idet_s_zakrytiya = {}

for i in range(1,200):
    
    count_x = 0
    
    for x in return_on_2:
        if x > i:
            count_x += 1
    
    for y in idet_since_gone:
        if y > i:
            count_x += 1
    
    idet_s_zakrytiya[i] = count_x

In [20]:
df = pd.DataFrame.from_dict(return_dict, orient='index')
df.to_excel('returned_on.xlsx', sheet_name='returned')

df2 = pd.DataFrame.from_dict(idet_s_zakrytiya, orient='index')
df2.to_excel('idet_s_zakrytiya.xlsx', sheet_name='idet')

In [81]:
# to find how many months customers are gone before becoming active again
# need return rate file data

months_between = []

for i in range(1, len(lines)):
    if lines[i][1] != '[]':
        months_between.extend(lines[i][1].replace('[', '').replace(']','').split(', '))

In [82]:
months_between_int = list(map(lambda x: int(x), months_between))

In [83]:
months_between_int.sort()

In [84]:
months_between_dict = {}

for i in range(183):
    months_between_dict[i] = months_between_int.count(i)

In [85]:
df3 = pd.DataFrame.from_dict(months_between_dict, orient='index')
df3.to_excel('months_between_dict2.xlsx', sheet_name='months_between')

# SWITCH TO PREMIUM CARD

In [98]:
file = open('gold_to_premium_card.csv')
csv_data = csv.reader(file,delimiter = ';')

In [99]:
lines = list(csv_data)
lines[0]

['CUSTOMER_RK',
 'AGREEMENT_RK',
 'CONTRACT_ID',
 'CARD_ACCOUNT_NUM',
 'CARD_NUM',
 'CARD_TYPE_CD',
 'CARD_TYPE_NM',
 'OPEN_DT',
 'OPEN_MDY',
 'CLOSE_DT',
 'CLOSE_MDY',
 'STATUS_CD',
 'PRODUCT_NAME',
 'CLOSED_ON_NTH_MONTH',
 'LIVING_THRU_CENS']

In [100]:
lines[0].extend(['CARD_COUNT', 'G_OR_P_FLAG', 'BREAK_MONTHS', 'BREAK_COUNT', 'MIN_OPEN', 'MAX_CLOSE'])

for i in range(1, len(lines)):
           
    if lines[i][0] != lines[i-1][0]:
        card_count = 1
        g_or_p_flag = []
        break_count = 0
        break_months = []
        min_open = 0
        max_close = 0
        
        if lines[i][10] == '' and lines[i][11] == 'Card OK':
            max_close = ''
        elif lines[i][10] == '':
            max_close = int(lines[i+1][8])
        else:
            max_close = int(lines[i][10])
        
        min_open = int(lines[i][8])
    
    else:
        card_count += 1
                        
        if max_close != '':
            
            if int(lines[i][8]) > max_close and lines[i][10] != '':
                break_months.append(int(round((int(lines[i][8]) - max_close) / 30, 0)))
                break_count += 1
                max_close = int(lines[i][10])
            elif lines[i][10] == '' and lines[i][11] == 'Card OK':
                max_close = ''
            elif lines[i][10] != '' and int(lines[i][10]) > max_close:
                max_close = int(lines[i][10])
            else:
                pass
    
    if 'PLATINUM' in lines[i][12] or 'INFINITE' in lines[i][12] or 'BLACK EDITION' in lines[i][12]:
        g_or_p_flag.append(1)
    else:
        g_or_p_flag.append(0)
    
    lines[i].append(card_count)
    lines[i].append(g_or_p_flag)
    lines[i].append(break_months)
    lines[i].append(break_count)
    lines[i].append(str(min_open))
    if max_close == '':
        lines[i].append(max_close)
    else:
        lines[i].append(str(max_close))

In [101]:
df = pd.DataFrame(lines)
df.columns = df.iloc[0]
df = df.iloc[1:]
df.to_csv('lines_g_or_p_flag.csv', index = False)

# MONTHS BETWEEN TXNS

In [157]:
file = open('debit_client_transactions_by_months.csv', 'r')
csv_data = csv.reader(file, delimiter=';')

In [158]:
lines = list(csv_data)

In [160]:
lines[1]

['9529264',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '1',
 '1',
 '1',
 '1',
 '1',
 '',
 '',
 '',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '',
 '1',
 '',
 '',
 '1',
 '',
 '',
 '',
 '',
 '1',
 '1',
 '1',
 '',
 '',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '',
 '1',
 '',
 '1',
 '1',
 '',
 '1',
 '1',
 '',
 '',
 '',
 '',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '',
 '1',
 '1',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1']

In [176]:
len(lines[1])

183

In [185]:
some_list = []
count_no_txn = 0
i = 1

while lines[1][i] == '':
    i += 1
    
else:

    for x in range(i,183):
        print(x)
        if x == len(lines[1]) and lines[1][len(lines[1])] == '':
            pass

        elif lines[1][i] == '':
            count_no_txn += 1

        elif lines[1][i] == '1':

            if count_no_txn > 0:
                some_list.append(count_no_txn)

            count_no_txn = 0


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182


In [175]:
some_list

[]

# FORECASTING NUMBER OF DEBIT CUSTOMERS

In [55]:
# need total clients from to_compute_hazards_all_client_cards_21.12.21

ratio_dict = {}
for i in range(1,125):
    try:
        result = 1 - gone_on_dict[i] / (survived_to_dict[i] + gone_on_dict[i])
    except:
        result = 1
    ratio_dict[i] = result

In [56]:
ratio_dict

{1: 0.9562327830324979,
 2: 0.9615465968620116,
 3: 0.9626474984244583,
 4: 0.9640812225087214,
 5: 0.9592377738715939,
 6: 0.9709768789591493,
 7: 0.9733963649541664,
 8: 0.9741861031827616,
 9: 0.9749646389111103,
 10: 0.9757203386018729,
 11: 0.9755964277034108,
 12: 0.9739047320187769,
 13: 0.9749708538574214,
 14: 0.9757019867320968,
 15: 0.9762074568200464,
 16: 0.9773533997545144,
 17: 0.9782150213085035,
 18: 0.9793957735822497,
 19: 0.9801660099541675,
 20: 0.9792568803173594,
 21: 0.9802249107285615,
 22: 0.9806807808827486,
 23: 0.9814400939488561,
 24: 0.9814018049534224,
 25: 0.9793640868936893,
 26: 0.9796555449634573,
 27: 0.9808660491298545,
 28: 0.9827925579109734,
 29: 0.982467778886562,
 30: 0.9835210504823496,
 31: 0.9840280843245565,
 32: 0.9840404623567419,
 33: 0.9840747155972616,
 34: 0.9829661392872986,
 35: 0.9827733949945593,
 36: 0.9816046705822088,
 37: 0.9803162816482242,
 38: 0.9844397828701051,
 39: 0.985603214747812,
 40: 0.9855638409734876,
 41: 0.9866

In [57]:
file = open('forecast_active_clients_nov_2021_(including_nov).csv')
csv_data = csv.reader(file,delimiter = ';')
lines = list(csv_data)

In [58]:
len(lines)

318581

In [59]:
lines[0]

['CUSTOMER_RK',
 'BREAK_MONTHS',
 'BREAK_COUNT',
 'MIN_OPEN',
 'MAX_TXN',
 'CLOSED_ON_NTH_MONTH',
 'LIVING_THRU_CENS',
 'ALIVE_FLG']

In [60]:
status_0 = {}
status_1 = {}
status_2 = {}
status_3 = {}
status_4 = {}
status_5 = {}

n_months_active_dec = []
for item in lines:
    try:
        n_months_active_dec.append(int(item[-2]))
    except:
        pass

In [61]:
for i in range(0,121):
    status_0[i] = n_months_active_dec.count(i)

In [62]:
status_0

{0: 9094,
 1: 11780,
 2: 11724,
 3: 6906,
 4: 6325,
 5: 6835,
 6: 5727,
 7: 5815,
 8: 5578,
 9: 5771,
 10: 4813,
 11: 5611,
 12: 6159,
 13: 7925,
 14: 5609,
 15: 3552,
 16: 3525,
 17: 3812,
 18: 3277,
 19: 3312,
 20: 5373,
 21: 6261,
 22: 5388,
 23: 4740,
 24: 7530,
 25: 9174,
 26: 6987,
 27: 4850,
 28: 4247,
 29: 4219,
 30: 4323,
 31: 4285,
 32: 5217,
 33: 5867,
 34: 5071,
 35: 3996,
 36: 5589,
 37: 5168,
 38: 3698,
 39: 2972,
 40: 2807,
 41: 2666,
 42: 2940,
 43: 2392,
 44: 2143,
 45: 2736,
 46: 2383,
 47: 1840,
 48: 1928,
 49: 2215,
 50: 2316,
 51: 1905,
 52: 1289,
 53: 1227,
 54: 1277,
 55: 1110,
 56: 1102,
 57: 1255,
 58: 1222,
 59: 1099,
 60: 1235,
 61: 1298,
 62: 1196,
 63: 1088,
 64: 1021,
 65: 708,
 66: 767,
 67: 768,
 68: 783,
 69: 539,
 70: 586,
 71: 692,
 72: 573,
 73: 913,
 74: 662,
 75: 563,
 76: 603,
 77: 1006,
 78: 1017,
 79: 703,
 80: 586,
 81: 1078,
 82: 659,
 83: 635,
 84: 655,
 85: 899,
 86: 677,
 87: 431,
 88: 699,
 89: 622,
 90: 502,
 91: 918,
 92: 561,
 93: 485,


In [282]:
status_1[1]= 10000*ratio_dict[1]
for i in range(2,122):
    status_1[i] = int(round(status_0[i-1] * ratio_dict[i-1]))

In [283]:
status_2[1]= 7000*ratio_dict[1]
for i in range(2,123):
    status_2[i] = int(round(status_1[i-1] * ratio_dict[i-1]))

In [284]:
status_3[1]= 9000*ratio_dict[1]
for i in range(2,124):
    status_3[i] = int(round(status_2[i-1] * ratio_dict[i-1]))

In [285]:
status_4[1]= 9000*ratio_dict[1]
for i in range(2,125):
    status_4[i] = int(round(status_3[i-1] * ratio_dict[i-1]))

In [286]:
status_5[1]= 9000*ratio_dict[1]
for i in range(2,126):
    status_5[i] = int(round(status_4[i-1] * ratio_dict[i-1]))

In [287]:
df = pd.DataFrame.from_dict(status_0, orient='index')
df.to_excel('status_0.xlsx', sheet_name='nov')

df = pd.DataFrame.from_dict(status_1, orient='index')
df.to_excel('status_1.xlsx', sheet_name='dec')

df = pd.DataFrame.from_dict(status_2, orient='index')
df.to_excel('status_2.xlsx', sheet_name='jan')

df = pd.DataFrame.from_dict(status_3, orient='index')
df.to_excel('status_3.xlsx', sheet_name='feb')

df = pd.DataFrame.from_dict(status_4, orient='index')
df.to_excel('status_4.xlsx', sheet_name='mar')

df = pd.DataFrame.from_dict(status_5, orient='index')
df.to_excel('status_5.xlsx', sheet_name='apr')


In [36]:
## NOT USED

fact_usage = []
for i in range(1,len(lines)):
    try:
        fact_usage.append(int(lines[i][-2]))
    except:
        fact_usage.append(0)

fact_usage_dict = {}
for x in range(max(fact_usage)+1):
    fact_usage_dict[int('{}'.format(x))] = fact_usage.count(x)
    

In [48]:
## NOT USED

started_using_on = {}

temp = []
for x in range(1, len(lines)):
    temp.append(int(lines[x][-1]))

for i in range(max(temp)+1):
    started_using_on[int('{}'.format(i))] = temp.count(i)

In [40]:
# SURVIVED TO (NTH MONTH WHEN CARD WAS SURELY ACTIVE)

df2 = pd.DataFrame.from_dict(survived_to_dict, orient='index')
df2.to_excel('survive_to.xlsx', sheet_name='survive_to')

In [41]:
# GONE ON (NTH MONTH WHEN CARD WAS CLOSED)

df3 = pd.DataFrame.from_dict(gone_on_dict, orient='index')
df3.to_excel('gone_on.xlsx', sheet_name='gone_on')

In [42]:
# FACT USAGE (FROM FIRST TO LAST TXN)

df4 = pd.DataFrame.from_dict(fact_usage_dict, orient='index')
df4.to_excel('fact_usage.xlsx', sheet_name='fact_usage')

In [49]:
# STARTED USING (ON NTH MONTH FIRST TXN)

df5 = pd.DataFrame.from_dict(started_using_on, orient='index')
df5.to_excel('started_using_on.xlsx', sheet_name='started_on')

In [116]:
#
#
# FOR ANNUAL SURVIVAL
#
#

file = open('annual_could_close_in_2021.csv')
csv_data = csv.reader(file,delimiter = ';')
lines = list(csv_data)

In [117]:
lines[0]

['CUSTOMER_RK',
 'BREAK_MONTHS',
 'BREAK_COUNT',
 'MIN_OPEN',
 'MAX_CLOSE',
 'CLOSED_ON_NTH_MONTH',
 'LIVING_THRU_CENS']

In [118]:
survived_to = []
for i in range(1,len(lines)):
    try:
        survived_to.append(int(lines[i][-1]))
    except:
        pass
    
gone_on = []
for i in range(1, len(lines)):
    try:
        gone_on.append(int(lines[i][-2]))
    except:
        pass

In [119]:
survived_to_dict = {}
for x in range(121):
    count_x = 0
    for item in survived_to:
        if item>=x:
            count_x += 1
        else:
            pass
    survived_to_dict[int('{}'.format(x))] = count_x

In [120]:
gone_on_dict = {}
for i in range(1, 121):
    gone_on_dict[int('{}'.format(i))] = gone_on.count(i)

In [121]:
df2 = pd.DataFrame.from_dict(survived_to_dict, orient='index')
df2.to_excel('survive_to.xlsx', sheet_name='survive_to')

df3 = pd.DataFrame.from_dict(gone_on_dict, orient='index')
df3.to_excel('gone_on.xlsx', sheet_name='gone_on')